In [1]:
from flask import Flask, request, render_template, flash, jsonify
import numpy as np
import pickle

# Load model and scalers
model = pickle.load(open('model.pkl', 'rb'))
sc = pickle.load(open('standscaler.pkl', 'rb'))
ms = pickle.load(open('minmaxscaler.pkl', 'rb'))

# Creating Flask app
app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Needed for flashing messages

# Define input ranges for each factor
input_ranges = {
    'Nitrogen': (0, 140),
    'Phosphorus': (5, 145),
    'Potassium': (5, 205),
    'Temperature': (8.83, 43.68),
    'Humidity': (14.26, 99.98),
    'Ph': (3.5, 9.94),
    'Rainfall': (20.21, 298.56)
}

@app.route('/')
def index():
    return render_template("index.html", ranges=input_ranges)

@app.route("/predict", methods=['POST'])
def predict():
    feature_list = []
    for factor, (min_val, max_val) in input_ranges.items():
        value = float(request.form[factor])
        if not min_val <= value <= max_val:
            flash(f"Warning: {factor} should be between {min_val} and {max_val}.", "warning")
            return render_template("index.html", ranges=input_ranges)

        feature_list.append(value)

    single_pred = np.array(feature_list).reshape(1, -1)

    try:
        scaled_features = ms.transform(single_pred)
        final_features = sc.transform(scaled_features)
        prediction = model.predict(final_features)
    except Exception as e:
        flash(f"An error occurred during prediction: {str(e)}", "danger")
        return render_template("index.html", ranges=input_ranges)

    crop_dict = {
        1: "Rice", 2: "Maize", 3: "Jute", 4: "Cotton", 5: "Coconut",
        6: "Papaya", 7: "Orange", 8: "Apple", 9: "Muskmelon",
        10: "Watermelon", 11: "Grapes", 12: "Mango", 13: "Banana",
        14: "Pomegranate", 15: "Lentil", 16: "Blackgram", 17: "Mungbean",
        18: "Mothbeans", 19: "Pigeonpeas", 20: "Kidneybeans", 21: "Chickpea",
        22: "Coffee"
    }

    crop = crop_dict.get(prediction[0], "Sorry, we could not determine the best crop to be cultivated with the provided data.")
    return jsonify({'prediction': crop})

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


c:\Users\Ramnivash\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Ramnivash\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
